In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mv drive/MyDrive/py_files/ /content

In [ ]:
import tensorflow as tf
from tensorflow import keras
import py_files
from py_files import load_data, const, bird_info
import warnings
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import  math
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [ ]:
f_drive = "drive/MyDrive/"

In [ ]:
# !unzip drive/MyDrive/img.zip

In [ ]:
image_size = (256, 256)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f_drive+"filtered_birds/",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
images, labels = load_data.load_data_from_folder(f_drive+"filtered_birds/")

In [ ]:
test_images, test_labels = load_data.load_data_from_folder(f_drive+"val_ten_uk_birds/")

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         image_rgb=cv2.cvtColor(images[i].numpy().astype("uint8"), cv2.COLOR_BGR2RGB)
#         plt.imshow(image_rgb)
#         plt.title(int(labels[i]))
#         plt.axis("off")

In [ ]:
images_np = np.array(images)
labels_np = np.array(labels)
# image_quality_np = np.array(image_quality)

In [ ]:
images_rgb =[]
for image in images:
  
  img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  images_rgb.append(img_rgb)

In [ ]:
test_images_brg =[]
for image in test_images:
  
  img_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  test_images_brg.append(img_rgb)

In [ ]:
pd_data=pd.DataFrame(columns=['Images', 'Labels', 'Image_Quality'])
pd_data['Images']=images
pd_data['Labels']=labels_np
# pd_data['Image_Quality']=image_quality_np

In [ ]:
pd_data.head()

In [ ]:
labels_value, labels_counts = np.unique(labels_np, return_counts=True)
# image_quality_values, image_quality_counts = np.unique(image_quality_np, return_counts=True)

In [ ]:
ia = cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB)
plt.imshow(ia)

In [ ]:
bird_info.visualise_10_10_images(images_rgb, labels, labels_value)

In [ ]:
bird_info.visualise_10_10_images(test_images_brg, test_labels, labels_value)

Models

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense, Dropout, Rescaling

def simple_model(shape):
    model = keras.Sequential()
    
    model.add(Conv2D(filters=6, kernel_size=(3, 3), strides=1, padding="valid", activation='relu', input_shape=shape))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=1, padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(units=320, activation='relu'))
    model.add(Dense(units=140, activation='relu'))
    model.add(Dense(units=60, activation='relu'))
    model.add(Dense(units=10, activation = 'softmax'))
    
    return model


In [ ]:
def deeper_model(shape):
    
    model = keras.Sequential()
    
    model.add(layers.RandomFlip("horizontal"))
    model.add(layers.RandomRotation(0.1))
    
    model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=1, kernel_regularizer=tf.keras.regularizers.l2(0.0005),padding="valid", activation='relu', input_shape=shape))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=1, kernel_regularizer=tf.keras.regularizers.l2(0.0005), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=56, kernel_size=(3, 3), strides=1, padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=86, kernel_size=(3, 3), strides=1, padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(units=320, activation='relu'))
    model.add(Dense(units=140, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=60, activation='relu'))

    model.add(Dense(units=10, activation = 'softmax'))
    return model

In [ ]:
# from tensorflow.keras.applications.resnet50 import ResNet50

# m=ResNet50(weights='imagenet')

In [ ]:
 m = deeper_model((256,256,3))

In [ ]:
m.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
# x1w = m.layers[5].get_weights()[0][:,:,0]

In [ ]:
# plt.subplots(figsize=(12,12))
# for i in range(1,16):
#     plt.subplot(3,5,i)
#     plt.imshow(x1w[:,:,i],interpolation="nearest",cmap="gray")
# plt.show()

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(images, labels,test_size=0.2, random_state=133)

In [ ]:
X_train=np.array(X_train).astype('float32')
Y_train=np.array(Y_train).astype('float32')
X_test=np.array(X_test).astype('float32')
Y_test=np.array(Y_test).astype('float32')

In [ ]:
Y_train = keras.utils.to_categorical(Y_train, num_classes=10)
Y_test = keras.utils.to_categorical(Y_test, num_classes=10)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelCheckpoint=ModelCheckpoint('alex_net1', save_weights_only=True, save_best_only=True)

In [ ]:
Y_train.shape

In [ ]:
history=m.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=200, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
X_val = np.array(test_images)
Y_val = keras.utils.to_categorical(test_labels, num_classes=10)

In [ ]:
Y_val.shape


In [ ]:
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

In [ ]:
m.evaluate(X_val, Y_val)

In [ ]:
preds = m.predict(X_val)

In [ ]:
preds[1]

In [ ]:
max_val_per_row=np.amax(preds, axis=1)
max_val_per_row

In [ ]:
max_where=[]
for index, max_row in enumerate(max_val_per_row):
  a = np.where(preds[index]==max_row)
  max_where.append(int(a[0]))

In [ ]:
from sklearn.metrics import confusion_matrix 
import seaborn as sns
results = confusion_matrix(test_labels, max_where)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(results, annot=True, fmt='d')


In [ ]:
plt.figure(figsize=(30, 30))
for index, image_l in enumerate(X_val):
  ax = plt.subplot(10, 10, index + 1)
  plt.imshow(X_val[index])
  plt.title(str(max_where[index]) +  "===" + str(data_test[index,1]))
  plt.axis('off')